Implement the following

Auto generate camel case aliases
The field type_ in our model is provided as type in source data, and should also serialize to type
The data we receive contains the following field names that need to map to our own model field names:
    number_of_doors is provided as doors
    manufactured_data is provided as completionDate
The field base_msrp_usd is provided as msrpUSD, and we want the serialization name to be baseMSRPUSD
We want the json serialized output of manufactered_date to be of pattern YYYY/MM/DD, but serializing to Python Dict should remain as date object.


In [42]:


from pydantic import BaseModel, ValidationError, ConfigDict, Field, field_serializer
from datetime import date
from enum import Enum
from pydantic.alias_generators import to_camel

class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"



data_json = '''
{
    "manufacturer": "BMW",
    "seriesName": "M4",
    "type": "Convertible",
    "isElectric": false,
    "completionDate": "2023-01-01",
    "msrpUSD": 93300,
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}
'''

expected_serialized_dict = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': AutomobileType.convertible,
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93300.0,
    'vin': '1234567890',
    'number_of_doors': 2,
    'registration_country': 'France',
    'license_plate': 'AAA-BBB'
}

expected_serialized_dict_by_alias = {
    'manufacturer': 'BMW',
    'seriesName': 'M4',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

expected_serialized_json_by_alias = (
    '{"manufacturer":"BMW","seriesName":"M4","type":"Convertible",'
    '"isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,'
    '"vin":"1234567890","numberOfDoors":2,"registrationCountry":"France",'
    '"licensePlate":"AAA-BBB"}'
)

In [43]:



class Automobile(BaseModel):
    model_config = ConfigDict(
        extra='forbid',
        str_strip_whitespace = True,
        validate_default = True,
        validate_assignment = True,
        alias_generator = to_camel,
    )
    manufacturer: str
    series_name: str = Field(alias='seriesName')
    type_: AutomobileType = Field(alias='type')
    is_electric: bool = Field(alias='isElectric')
    manufactured_date: date = Field(validation_alias='completionDate')
    base_msrp_usd: float = Field(alias='msrpUSD', serialization_alias='baseMSRPUSD')
    vin: str
    number_of_doors: int = Field(alias='doors', serialization_alias='numberOfDoors')
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")


In [44]:
car = Automobile.model_validate_json(data_json)

In [45]:
car.model_dump()

{'manufacturer': 'BMW',
 'series_name': 'M4',
 'type_': <AutomobileType.convertible: 'Convertible'>,
 'is_electric': False,
 'manufactured_date': datetime.date(2023, 1, 1),
 'base_msrp_usd': 93300.0,
 'vin': '1234567890',
 'number_of_doors': 2,
 'registration_country': 'France',
 'license_plate': 'AAA-BBB'}

In [46]:
assert car.model_dump() == expected_serialized_dict

In [47]:
car.model_dump(by_alias=True)

{'manufacturer': 'BMW',
 'seriesName': 'M4',
 'type': <AutomobileType.convertible: 'Convertible'>,
 'isElectric': False,
 'manufacturedDate': datetime.date(2023, 1, 1),
 'baseMSRPUSD': 93300.0,
 'vin': '1234567890',
 'numberOfDoors': 2,
 'registrationCountry': 'France',
 'licensePlate': 'AAA-BBB'}

In [48]:
car.model_dump(by_alias=True) == expected_serialized_dict_by_alias

True

In [49]:
 car.model_dump_json(by_alias=True)

'{"manufacturer":"BMW","seriesName":"M4","type":"Convertible","isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,"vin":"1234567890","numberOfDoors":2,"registrationCountry":"France","licensePlate":"AAA-BBB"}'

In [50]:
 car.model_dump_json(by_alias=True) == expected_serialized_json_by_alias

True

In [ ]:
expected_serialized_json_by_alias = (
    '{"manufacturer":"BMW","seriesName":"M4","type":"Convertible",'
    '"isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,'
    '"vin":"1234567890","numberOfDoors":2,"registrationCountry":"France",'
    '"licensePlate":"AAA-BBB"}'
)